In [1]:
from deltalake import DeltaTable

dt = DeltaTable("s3://delta/iot_poc_v1", None, {
    "aws_default_region": "us-east-2",
    "aws_secret_access_key": "minioadmin",
    "allow_http": "true",
    "aws_access_key_id": "minioadmin",
    "aws_endpoint": "http://localhost:9000"
})



In [2]:
dt.files()

['date=2023-01-26/part-00000-227c870b-6ea9-4f22-b604-c4fa17ec1cec-c000.snappy.parquet']

In [3]:
dt.to_pandas()

,poc_id,beacon_report,selected_witnesses,unselected_witnesses,date
0,"b' \xff?\x8f\x1c\x191\x9b\xf6\xd26n\xe2,\xd3\x...","{'received_timestamp': 1674765710996, 'locatio...","[{'received_timestamp': 1674765712656, 'status...","[{'received_timestamp': 1674765711909, 'status...",2023-01-26
1,b'\x1c;\x82\xcc\xb8\x87\xf5\xbf\x01F\x13\xe6\x...,"{'received_timestamp': 1674765711613, 'locatio...","[{'received_timestamp': 1674765712648, 'status...","[{'received_timestamp': 1674765711826, 'status...",2023-01-26
2,b'\x8b\xb6\xd3\xb3\x0bvu\xe2\x7fTq\xaam5K\xbe\...,"{'received_timestamp': 1674765709433, 'locatio...","[{'received_timestamp': 1674765709563, 'status...","[{'received_timestamp': 1674765711836, 'status...",2023-01-26
3,b'6\xf1\xc9\xect\xf1}^\x9buQ\x12Bo*\xb0y\xbds\...,"{'received_timestamp': 1674765712918, 'locatio...","[{'received_timestamp': 1674765709590, 'status...","[{'received_timestamp': 1674765711817, 'status...",2023-01-26
4,"b'|-2\x84Y\x01Q\xcd\x17\x17.\xcdQ2\n{R&-""T\xba...","{'received_timestamp': 1674765714222, 'locatio...","[{'received_timestamp': 1674765709596, 'status...","[{'received_timestamp': 1674765711781, 'status...",2023-01-26
...,...,...,...,...,...
14446,b'!\xab4\x1b\x15\x1e\xfe3\xa2\xad\x02\xa5\xe6\...,"{'received_timestamp': 1674766790476, 'locatio...","[{'received_timestamp': 1674765817669, 'status...","[{'received_timestamp': 1674765767833, 'status...",2023-01-26
14447,b'\xe5\xe7C\xe0B\x88\xb2\x85=B=\x19<v\xf4\x08V...,"{'received_timestamp': 1674766788086, 'locatio...","[{'received_timestamp': 1674765808212, 'status...","[{'received_timestamp': 1674765767790, 'status...",2023-01-26
14448,b'K^\x0b\xf4U\xf9\x05\x86\ti\x97G^\x16\xd8\xa9...,"{'received_timestamp': 1674766793774, 'locatio...","[{'received_timestamp': 1674765808076, 'status...","[{'received_timestamp': 1674765767768, 'status...",2023-01-26
14449,b'\x08K\xf3d\x8f\x85\xda\xf0\xdf\xd0i\x9c/\x1a...,"{'received_timestamp': 1674766791780, 'locatio...","[{'received_timestamp': 1674765808372, 'status...","[{'received_timestamp': 1674765767752, 'status...",2023-01-26
